In [ ]:
from dotenv import load_dotenv
from src.create_kb import create_kb_from_file
from langchain.chat_models import init_chat_model
from langgraph.graph import START, StateGraph
from typing_extensions import TypedDict

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

c:\Users\Kaden\OneDrive\Professional\Athena\Note Creation\RAG\virtual_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
# create knowledge base from transcript
file_path = "transcript.txt"
kb_id = "transcript"
kb = create_kb_from_file(kb_id, file_path)

Creating KB with id transcript...


In [3]:
# prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [4]:
# Define state for application
class State(TypedDict):
    question: str
    context: list[dict]
    answer: str

In [5]:
# Define application steps
def retrieve(state: State):
    search_queries = [state["question"]]
    retrieved_docs = kb.query(search_queries)
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc["content"] for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [6]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [12]:
response = graph.invoke({"question": "What are some papers to understand probability?"})
print(response["answer"])

To understand probability, you can refer to the document titled 'Introducing Probability,' which covers the fundamentals of probability theory, including definitions, axioms, and examples. Additionally, Peter Backus's entertaining paper on estimating dating probabilities provides a practical perspective on applying probabilistic arguments. For further exploration, you might also consider simulations and historical examples like De Méré’s Paradox discussed within the same document.
